In [1]:

import sys
sys.path.append('/home/chandy/gsoc/Darkpose_Tensorflow/lib/')
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()


In [2]:

import tensorflow as tf
from data.dataset_joint import JointsDataset
from data import coco
from data import coco_data
from config import cfg

from models import pose_resnet

from tensorflow.keras.losses import binary_crossentropy
import tensorflow.keras.backend as K



In [3]:
import argparse

from config import cfg
from config import update_config

def mse(y_true, y_pred, sample_weight):
    error = y_true-y_pred
    out=tf.mean(tf.sqrt(error)*sample_weight)
    return tf.cast(out, dtype='float64')

def dice_loss(y_true, y_pred):
    intersection = tf.reduce_sum(tf.multiply(y_true, y_pred))
    union = tf.reduce_sum(tf.square(y_true)) + tf.reduce_sum(tf.square(y_pred))
    loss = 1. - 2 * intersection / (union + K.epsilon())
    return loss


In [4]:

data_3 = coco.COCODataset(cfg, cfg.DATASET.ROOT, cfg.DATASET.TRAIN_SET, True)



# data=tf.data.Dataset.from_tensor_slices(data_3)

loading annotations into memory...
Done (t=4.87s)
creating index...
index created!
loading annotations into memory...
Done (t=4.82s)
creating index...
index created!


In [5]:
# features_shape = [None, 256, 256, 3]
# labels_shape = [None, 17,64,64]
# labels_shape_1 = [None, 17]
# ds = tf.data.Dataset.from_generator(
#         data_3, (tf.float32, tf.float32,tf.float32),
#         (tf.TensorShape(features_shape), tf.TensorShape(labels_shape),tf.TensorShape(labels_shape_1)))

In [6]:
# import numpy as np
# for images, target in data_3:
#     print(target.shape)
# # #     print(target)
#     print(target_weights)
#     print(meta)
# images, target, target_weights,meta = data_3

In [7]:

# model = eval('models.'+cfg.MODEL.NAME+'.get_pose_net')(
#         cfg, is_train=True
#     )

model= pose_resnet.get_pose_net(
        cfg, is_train=True
    )
model.build(input_shape=(None,256, 256, 3))
model.compile(loss=dice_loss, optimizer='adam')
model.summary()
model.fit_generator(generator=data_3,epochs=2,
                    validation_data=data_3)

Instructions for updating:
Colocations handled automatically by placer.
Model: "pose_res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential_1 (Sequential)    multiple                  218624    
_________________________________________________________________
sequential_3 (Sequential)    multiple                  1226752   
________________________________________________

/home/chandy/anaconda3/envs/gsoc/lib/python3.7/site-packages/tensorflow/python/keras/engine/training_v1.py:1246: UserWarning: `model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`model.fit_generator` is deprecated and '


ValueError: Variable <tf.Variable 'conv2d/kernel:0' shape=(7, 7, 3, 64) dtype=float32> has `None` for gradient. Please make sure that all of your ops have a gradient defined (i.e. are differentiable). Common ops without gradient: K.argmax, K.round, K.eval.

In [ ]:
# for images, target, target_weights,meta in data_3:
#     print(images.shape)
print(data_3)
# image, target = data_3
# model.fit_generator(
#     data_3,
#     # samples_per_epoch=train_samples,
   
#     epochs=2,
#     validation_data=data_3)

# # for images, target, target_weight, meta in train_data_iter:
# #     print(images.shape)
    